# Setup

In [1]:
# Packages -----------------------------------------------#

# Data Analysis
import xarray as xr
import numpy as np
import pandas as pd
import metpy.calc as mpcalc
import matplotlib.dates as dates
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# Timing Processes and Progress
import time
from tqdm import tqdm

# make sure the figures plot inline rather than at the end
%matplotlib inline

# Parameters

In [2]:
binwidth = 0.25

# Read Data

In [3]:
# get data from tigress
slainfn = 'sla_processed.nc'

ds = xr.open_dataset(slainfn)
ds

<xarray.Dataset>
Dimensions:             (lat: 200, latitude: 200, lon: 340, longitude: 340, month: 12, month_time: 315, season: 4, season_time: 106, time: 9563)
Coordinates:
  * season_time         (season_time) datetime64[ns] 1993-12-01 ... 2020-03-01
  * longitude           (longitude) float64 35.12 35.38 35.62 ... 119.6 119.9
  * latitude            (latitude) float64 -19.88 -19.62 -19.38 ... 29.62 29.88
  * lon                 (lon) float64 35.12 35.38 35.62 ... 119.4 119.6 119.9
  * lat                 (lat) float64 -19.88 -19.62 -19.38 ... 29.38 29.62 29.88
  * time                (time) datetime64[ns] 1994-01-01 ... 2020-03-07
  * month_time          (month_time) datetime64[ns] 1994-01-01 ... 2020-03-01
  * month               (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * season              (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    daily_sla           (time, lat, lon) float64 ...
    mon_sla             (month_time, lat, lon) float64 ...
    seas_sla          

In [4]:
def latlon_coarsen(invar,lat,lon,bounds,binwidth = 1):
    import numpy as np
 
    # create a pandas dataframe
    df = pd.DataFrame(dict(
            invar = np.array(invar),
            lat= np.array(lat),
            lon= np.array(lon)
        ))
    
    # create 1 degree bins
    latedges = np.arange(bounds[2]-(binwidth/2),bounds[3]+(binwidth/2),binwidth)
    lat_inds = list(range(len(latedges)-1))

    lonedges = np.arange(bounds[0]-(binwidth/2),bounds[1]+(binwidth/2),binwidth)
    lon_inds = list(range(len(lonedges)-1))

    latbins = latedges[1:]-(binwidth/2)
    lonbins = lonedges[1:]-(binwidth/2)

    df['latedges'] = pd.cut(lat, latedges)
    df['lonedges'] = pd.cut(lon, lonedges)
    df['latbins_ind'] = pd.cut(lat, latedges,labels = lat_inds)
    df['lonbins_ind'] = pd.cut(lon, lonedges,labels = lon_inds)
    df['lat_lon_indx']=df.groupby(['latbins_ind', 'lonbins_ind']).ngroup()
    grouped = df.groupby(['latbins_ind', 'lonbins_ind'])

    invar_BINNED = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray)
    invar_BINNED[:] = np.nan

    invar_binned_ave = np.zeros((len(latbins),len(lonbins)), dtype=np.ndarray)
    invar_binned_ave[:] = np.nan


    #extract the data for each group
    for name, group in grouped:
        i = np.array(group.latbins_ind)
        j = np.array(group.lonbins_ind)

        invar_BINNED[i[0],j[0]] = group.invar

        invar_binned_ave[i[0],j[0]] = np.nanmean(group.invar)

    return np.array(invar_binned_ave,dtype = float),lonbins,latbins

# Put into 0.25 degree bins

In [ ]:
# grid lat lon data
xx,yy = np.meshgrid(ds.lon,ds.lat)

# define extent of grid
bounds = [35,120,-20,30]

_,lonbins,latbins =latlon_coarsen(np.array(ds.mon_sla_mon_anom[0,:,:]).ravel(),yy.ravel(),xx.ravel(),bounds,binwidth)

# monthly data
mon_sla_binned = np.zeros([ds.mon_sla.shape[0],len(latbins),len(lonbins)])
for tt in range(ds.mon_sla.shape[0]):
    mon_sla_binned[tt,:,:],_,_= latlon_coarsen(np.array(ds.mon_sla[tt,:,:]).ravel(),
                                               yy.ravel(),xx.ravel(),bounds,binwidth)

# monthly climatology
sla_mon_clim_binned=np.zeros([ds.sla_mon_clim.shape[0],len(latbins),len(lonbins)])
for tt in range(ds.sla_mon_clim.shape[0]):
    sla_mon_clim_binned[tt,:,:],_,_ = latlon_coarsen(np.array(ds.sla_mon_clim[tt,:,:]).ravel(),
                                                   yy.ravel(),xx.ravel(),bounds,binwidth)

# monthly anomaly
mon_sla_mon_anom_binned=np.zeros([ds.mon_sla_mon_anom.shape[0],len(latbins),len(lonbins)])
for tt in range(ds.mon_sla_mon_anom.shape[0]):
    mon_sla_mon_anom_binned[tt,:,:],_,_= latlon_coarsen(np.array(ds.mon_sla_mon_anom[tt,:,:]).ravel(),
                                                      yy.ravel(),xx.ravel(),bounds,binwidth)

# seasonal climatology
sla_seas_clim_binned=np.zeros([ds.sla_seas_clim.shape[0],len(latbins),len(lonbins)])
for tt in range(ds.sla_seas_clim.shape[0]):
    sla_seas_clim_binned[tt,:,:],_,_ = latlon_coarsen(np.array(ds.sla_seas_clim[tt,:,:]).ravel(),
                                                    yy.ravel(),xx.ravel(),bounds,binwidth)

# seasonal anomaly
seas_sla_seas_anom_binned=np.zeros([ds.seas_sla_seas_anom.shape[0],len(latbins),len(lonbins)])
for tt in range(ds.seas_sla_seas_anom.shape[0]):
    seas_sla_seas_anom_binned[tt,:,:],_,_ = latlon_coarsen(np.array(ds.seas_sla_seas_anom[tt,:,:]).ravel(),
                                                         yy.ravel(),xx.ravel(),bounds,binwidth)
    
    

/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mea

/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mea

/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mean of empty slice
/home/jennap/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: Mea

In [ ]:
fig = plt.figure(figsize=(15, 4))
ax = fig.add_subplot(1,3,1)
plt.pcolormesh(xx,yy,ds.mon_sla_mon_anom[0,:,:],cmap=plt.cm.PuOr_r)

ax = fig.add_subplot(1,3,2)
plt.pcolormesh(lonbins,latbins,mon_sla_mon_anom_binned[0,:,:],cmap=plt.cm.PuOr_r)

# ax = fig.add_subplot(1,3,3)
# plt.pcolormesh(lonbins_2deg,latbins_2deg,mon_sla_mon_anom_dtrnd_2deg[0,:,:],cmap=plt.cm.PuOr_r)

In [ ]:
# convert to xarray dataset
ds_SLA=xr.Dataset(coords={'lon': lonbins,
                    'lat': latbins,
                    'time': ds.time})

# add variables to dataset

ds_SLA["sla_mon_clim"]=xr.DataArray(sla_mon_clim_binned,dims = ['month','lat', 'lon'],
                     coords =[ds.month,latbins,lonbins])
ds_SLA["sla_seas_clim"]=xr.DataArray(sla_seas_clim_binned,dims = ['season','lat', 'lon'],
                     coords =[ds.season,latbins,lonbins])

# anom
ds_SLA["mon_sla_mon_anom"]=xr.DataArray(mon_sla_mon_anom_binned,dims = ['month_time','lat', 'lon'],
                     coords =[ds.month_time,latbins,lonbins])
ds_SLA["seas_sla_seas_anom"]=xr.DataArray(seas_sla_seas_anom_binned,dims = ['season_time','lat', 'lon'],
                     coords =[ds.season_time,latbins,lonbins])

ds_SLA.to_netcdf('sla_' + str(binwidth) + '_degree_binned.nc',mode='w',format = "NETCDF4")

ds_SLA